## Deep riccy project

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! tar -zxvf /content/drive/MyDrive/Uni/DeepRiccy/refcocog.tar.gz

In [2]:
#set local path to the data

#this is for colab
#local_path = '/content/refcocog/images/'
#local_annotations = '/content/refcocog/annotations/'

#this is for local
local_path = './refcocog/images/' 
local_annotations = './refcocog/annotations/' 

In [3]:
#import section
import json
import pickle
import torch
from PIL import Image
from torch.utils.data import Dataset

In [2]:
#class creation for the items
class CocogImage():
    def __init__(self, id, json_file, referece):
        self.id = id
        self.data = json_file['images'][id]
        self.bboxes = []
        for i in json_file:
            if i['image_id'] == self.id:
                self.bboxes.append(i)
        self.annotations = []
        
        for i in referece:
            if i['image_id'] == self.id:
                self.annotations.append(i)


    
    def get_image(self, transform = transforms.Compose([transforms.PILToTensor()])):
        img = Image.open(local_path + self.data['file_name'])
        return transform(img)
    
    def id(self):
        return self.data['id']

    def width(self):
        return self.data['width']

    def height(self):
        return self.data['height']

    def bbox(self):
        return self.bboxes
    
    def annotations(self):
        return self.annotations

{"license": 1, 
"file_name": "COCO_train2014_000000262528.jpg",
"coco_url": "http://mscoco.org/images/262528", 
"height": 480, 
"width": 318, 
"date_captured": "2013-11-19 19:02:15", 
"flickr_url": "http://farm8.staticflickr.com/7271/7135360435_bcc00e3f5f_z.jpg", 
"id": 262528}

{"segmentation": [[116.81, 102.77, 121.15, 95.96, 114.95, 54.45, 111.24, 33.7, 119.29, 9.34, 129.82, 3.45, 135.4, 0.04, 156.47, 0.27, 153.27, 3.3, 155.4, 5.51, 158.02, 5.83, 159.82, 8.04, 161.05, 8.21, 166.29, 15.49, 168.34, 21.22, 168.25, 25.23, 172.1, 36.82, 170.22, 39.36, 170.38, 44.84, 175.21, 54.22, 175.79, 64.53, 176.36, 68.05, 177.83, 84.14, 178.24, 97.86, 174.72, 100.8, 170.38, 100.47, 169.89, 97.94, 172.35, 95.97, 172.59, 93.43, 171.37, 91.8, 171.86, 95.64, 168.42, 96.71, 167.44, 97.77, 166.45, 105.71, 164.49, 110.87, 162.36, 119.35, 163.02, 127.86, 160.23, 131.54, 158.84, 137.73, 159.74, 142.31, 161.46, 145.67, 161.05, 151.15, 161.18, 168.79, 156.33, 172.25, 145.08, 167.75, 133.32, 165.85, 124.32, 164.12, 126.05, 154.95, 122.76, 142.49, 123.63, 134.7, 124.49, 127.09, 120.34, 127.61, 115.49, 124.49, 114.8, 119.82, 115.01, 115.03, 115.71, 112.35, 116.79, 111.46, 113.35, 111.01, 112.2, 109.04, 114.82, 102.26]],
"area": 8243.56785, 
"iscrowd": 0, 
"image_id": 191327, 
"bbox": [111.24, 0.04, 67.0, 172.21], 
"category_id": 1, 
"id": 197178}, 

In [37]:
class Coco(Dataset):
  def __init__(self, size):
    super(Coco, self).__init__()
    self.size = size
    #load json file
    with open(local_annotations + 'instances.json', 'r') as f:
      self.data = json.load(f)
    #load references(umd)
    with open(local_annotations + 'refs(umd).p', 'rb') as f:
      self.references = pickle.load(f)
      f.close()
    #load references(google)
    with open(local_annotations + 'refs(google).p', 'rb') as f:
      self.references_google = pickle.load(f)
      f.close()
    
    
  def __getitem__(self, idx):
    if idx >= self.size:
      raise IndexError()
    #import image form json
    image = CocogImage(idx, self.data, self.references)
    
    return image.get_image(), image.bbox(), image.annotations()

  def __len__(self):
    return self.size
